In [1]:
import csv
import ast

class Player:
    def __init__(self, nick):
        self.nick = nick
        self.elo = 1000
        self.matches_played = 0
        self.wins = 0
        self.losses = 0

    def win_rate(self):
        if self.matches_played == 0:
            return 0
        return self.wins / self.matches_played


players_temp = {}
PLAYERS = []

with open("matches.csv", newline="", encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile)

    players_unique = set()

    for row in reader:
        players_a = row["a"]
        players_a = ast.literal_eval(players_a)

        players_b = row["b"]
        players_b = ast.literal_eval(players_b)

        for player in players_a:
            players_unique.add(player)

        for player in players_b:
            players_unique.add(player)
        
for player in players_unique:
    players_temp[player] = Player(player)


for player in players_temp:
    PLAYERS.append(players_temp[player])


print(f"{len(PLAYERS)} graczy")

18 graczy


In [2]:
class MatchBasic:
    def __init__(self, match_id, winners, losers, date):
        self.match_id = match_id
        self.winners = winners  # lista playerow
        self.losers = losers    # lista playerow
        self.date = date

class MatchPlayer:
    def __init__(self, match_id, player, elo_change, is_winner):
        self.match_id = match_id
        self.player = player    # ref do Player
        self.elo_change = elo_change
        self.elo_before = 0
        self.elo_after = 0
        self.is_winner = is_winner

class Match(MatchBasic):
    def __init__(self, match_id, winners, losers, date, match_players=None):
        super().__init__(match_id, winners, losers, date)
        self.match_players = match_players or [] 

def get_player_by_nick(nick) -> Player:
    for player in PLAYERS:
        if player.nick == nick:
            return player
    return None

with open("matches.csv", newline="", encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile)
    matches = []

    for row in reader:
        match_id = row["id"]
        winner_team = row["winner"]
        date = row["date"]
        
        def safe_parse_list(s):
            try:
                return ast.literal_eval(s)
            except (ValueError, SyntaxError):
                import re
                matches = re.findall(r"'([^']*)'", s)
                if matches:
                    return matches
                return [s.strip("[]")]
        
        team_a_nicks = safe_parse_list(row["a"])
        team_b_nicks = safe_parse_list(row["b"])
        
        team_a_players = [get_player_by_nick(nick) for nick in team_a_nicks]
        team_b_players = [get_player_by_nick(nick) for nick in team_b_nicks]

        if winner_team == "a":
            winners = team_a_players
            losers = team_b_players
        else:
            winners = team_b_players
            losers = team_a_players
        
        match_players = []

        for player in winners:
            player.matches_played += 1
            player.wins += 1
            match_players.append(MatchPlayer(match_id, player, 0, True))
            
        for player in losers:
            player.matches_played += 1
            player.losses += 1
            match_players.append(MatchPlayer(match_id, player, 0, False))
        
        match = Match(match_id, winners, losers, date, match_players)
        matches.append(match)

for match in matches[:3]:
    winners_nicks = [player.nick for player in match.winners]
    losers_nicks = [player.nick for player in match.losers]
    print(f"{match.match_id}: {winners_nicks} vs {losers_nicks}")
    
    print("  Gracze w meczu:")
    for mp in match.match_players:
        player = mp.player
        status = "Wygrał" if mp.is_winner else "Przegrał"
        print(f"    {player.nick}:\t{status} (W: {player.wins}, L: {player.losses}, M: {player.matches_played})")
    print()

print(f"{len(matches)} meczy")
print(f"{len(PLAYERS)} graczy")

# print("\nNajlepsi gracze (liczba wygranych):")
# top_winners = sorted(PLAYERS, key=lambda p: p.wins, reverse=True)[:10]
# for i, player in enumerate(top_winners, 1):
#     win_rate = round(player.win_rate() * 100, 1) if player.matches_played > 0 else 0
#     print(f"{i}. {player.nick}: {player.wins} wygranych / {player.matches_played} meczów ({win_rate}%)")

67dd3f652aef101c279ab864: ['Filip Rusztowicz', 'kb316244', 'kondrix', 'rafał radosław malczyk'] vs ['filip_pro', 'marta', 'suchix', 'topol']
  Gracze w meczu:
    Filip Rusztowicz:	Wygrał (W: 23, L: 11, M: 34)
    kb316244:	Wygrał (W: 4, L: 1, M: 5)
    kondrix:	Wygrał (W: 4, L: 4, M: 8)
    rafał radosław malczyk:	Wygrał (W: 25, L: 11, M: 36)
    filip_pro:	Przegrał (W: 41, L: 59, M: 100)
    marta:	Przegrał (W: 10, L: 23, M: 33)
    suchix:	Przegrał (W: 72, L: 63, M: 135)
    topol:	Przegrał (W: 12, L: 14, M: 26)

67dd39972aef101c279ab863: ['Filip Rusztowicz', 'kb316244', 'kondrix', 'rafał radosław malczyk'] vs ['filip_pro', 'marta', 'suchix', 'topol']
  Gracze w meczu:
    Filip Rusztowicz:	Wygrał (W: 23, L: 11, M: 34)
    kb316244:	Wygrał (W: 4, L: 1, M: 5)
    kondrix:	Wygrał (W: 4, L: 4, M: 8)
    rafał radosław malczyk:	Wygrał (W: 25, L: 11, M: 36)
    filip_pro:	Przegrał (W: 41, L: 59, M: 100)
    marta:	Przegrał (W: 10, L: 23, M: 33)
    suchix:	Przegrał (W: 72, L: 63, M: 135)

In [3]:
def calculate_elo_ratings(matches, K_SOLO=38, K_TEAM=24):
    def expected_score(rating_a, rating_b):
        return 1 / (1 + 10 ** ((rating_b - rating_a) / 400))
    
    def calculate_new_rating(old_rating, expected, actual, k_factor):
        return old_rating + k_factor * (actual - expected)
        
    for match in matches:
        for mp in match.match_players:
            mp.elo_before = mp.player.elo
            
        is_team_match = len(match.winners) > 1 or len(match.losers) > 1
        k_factor = K_TEAM if is_team_match else K_SOLO
        
        winners_avg_elo = sum(player.elo for player in match.winners) / len(match.winners)
        losers_avg_elo = sum(player.elo for player in match.losers) / len(match.losers)
        
        winners_expected = expected_score(winners_avg_elo, losers_avg_elo)
        losers_expected = 1 - winners_expected
        
        team_size_factor = 1.0
        if is_team_match:
            team_size = max(len(match.winners), len(match.losers))
            team_size_factor = 1.0 / (1 + 0.1 * (team_size - 2))  

        adjusted_k = k_factor * team_size_factor
        
        for mp in match.match_players:
            if mp.is_winner:
                old_elo = mp.player.elo
                new_elo = calculate_new_rating(old_elo, winners_expected, 1.0, adjusted_k)
                elo_change = new_elo - old_elo
            else:
                old_elo = mp.player.elo
                new_elo = calculate_new_rating(old_elo, losers_expected, 0.0, adjusted_k)
                elo_change = new_elo - old_elo
            
            mp.elo_change = round(elo_change, 1)
            mp.player.elo = round(new_elo, 1)
    
    return matches

matches = matches[::-1]

calculated_matches = calculate_elo_ratings(matches)

sorted_players = sorted(PLAYERS, key=lambda p: p.elo, reverse=True)
print("\nTopka:")
for i, player in enumerate(sorted_players):
    print(f"{i+1}. {player.nick}: {player.elo} ({player.wins}/{player.matches_played} matches)")


Topka:
1. rafał radosław malczyk: 1082.0 (25/36 matches)
2. michal: 1071.8 (27/42 matches)
3. Filip Rusztowicz: 1043.3 (23/34 matches)
4. remzak: 1037.7 (56/118 matches)
5. suchix: 1037.2 (72/135 matches)
6. karol: 1035.1 (17/29 matches)
7. TenOdHighGrounda: 1024.9 (2/3 matches)
8. kb316244: 1021.9 (4/5 matches)
9. egzorcysta: 1012.0 (3/5 matches)
10. tirstus: 1009.3 (3/5 matches)
11. ż: 1002.3 (1/3 matches)
12. kondrix: 993.7 (4/8 matches)
13. piloniak: 978.0 (3/9 matches)
14. topol: 967.8 (12/26 matches)
15. jacek: 964.9 (0/2 matches)
16. marta: 925.4 (10/33 matches)
17. oskarito: 898.7 (24/61 matches)
18. filip_pro: 894.0 (41/100 matches)


In [4]:
print("\nHistoria meczów (od najstarszego do najnowszego):")
for match in matches:
    winners_str = ", ".join(p.nick for p in match.winners)
    losers_str = ", ".join(p.nick for p in match.losers)
    match_date = match.date[:10] # tylko data, bez czasu
    print(f"\nMecz {match_date}: {winners_str} vs {losers_str}")
    
    print("Zwycięzcy:")
    for mp in [mp for mp in match.match_players if mp.is_winner]:
        print(f"  {mp.player.nick}: {mp.elo_before} -> {mp.elo_before + mp.elo_change} (+{mp.elo_change})")
    
    print("Przegrani:")
    for mp in [mp for mp in match.match_players if not mp.is_winner]:
        print(f"  {mp.player.nick}: {mp.elo_before} -> {mp.elo_before + mp.elo_change} ({mp.elo_change})")


Historia meczów (od najstarszego do najnowszego):

Mecz 2024-12-10: remzak vs suchix
Zwycięzcy:
  remzak: 1000 -> 1019.0 (+19.0)
Przegrani:
  suchix: 1000 -> 981.0 (-19.0)

Mecz 2024-12-10: remzak vs suchix
Zwycięzcy:
  remzak: 1019.0 -> 1035.9 (+16.9)
Przegrani:
  suchix: 981.0 -> 964.1 (-16.9)

Mecz 2024-12-10: suchix vs remzak
Zwycięzcy:
  suchix: 964.1 -> 987.0 (+22.9)
Przegrani:
  remzak: 1035.9 -> 1013.0000000000001 (-22.9)

Mecz 2024-12-10: filip_pro vs remzak
Zwycięzcy:
  filip_pro: 1000 -> 1019.7 (+19.7)
Przegrani:
  remzak: 1013.0 -> 993.3 (-19.7)

Mecz 2024-12-10: filip_pro vs remzak
Zwycięzcy:
  filip_pro: 1019.7 -> 1037.3 (+17.6)
Przegrani:
  remzak: 993.3 -> 975.6999999999999 (-17.6)

Mecz 2024-12-10: filip_pro vs remzak
Zwycięzcy:
  filip_pro: 1037.3 -> 1053.0 (+15.7)
Przegrani:
  remzak: 975.7 -> 960.0 (-15.7)

Mecz 2024-12-10: remzak vs filip_pro
Zwycięzcy:
  remzak: 960.0 -> 984.0 (+24.0)
Przegrani:
  filip_pro: 1053.0 -> 1029.0 (-24.0)

Mecz 2024-12-11: suchix vs to

# todo
- moze jakis smieszny wykresik do tego jeszcze dorzucic, na zmitacu i tutaj

# pomysły

customowe K dla dni tygodnia?
+jakis bonus dla dnia tygodnia w ktorym najrzadziej sie gra

cos trzeba wymyslec hehe